In [1]:
import numpy as np 
import pandas as pd 
from tensorflow.keras.models import *
from tensorflow.keras.layers import *  
from tensorflow.keras.callbacks import *
from tqdm import tqdm
import time
import random
import math
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from tensorflow_probability import distributions as tfd

# Load Data

In [2]:
## season 2 dataframes 
train_x_2 = pd.read_csv('train_x_df.csv')
train_y_2 = pd.read_csv('train_y_df.csv') 
test_x_2 = pd.read_csv('test_x_df.csv') 
submission = pd.read_csv('sample_submission.csv') 

train_x_2.shape, train_y_2.shape, test_x_2.shape, submission.shape


((10572180, 12), (919320, 12), (738300, 12), (535, 3))

In [3]:
def df2d_to_array3d(df_2d):
    feature_size = df_2d.iloc[:,2:].shape[1]
    time_size = len(df_2d.time.value_counts())
    sample_size = len(df_2d.sample_id.value_counts())
    array_3d = df_2d.iloc[:,2:].values.reshape([sample_size, time_size, feature_size])
    return array_3d


x_train = df2d_to_array3d(train_x_2) 
y_train = df2d_to_array3d(train_y_2) 
x_test = df2d_to_array3d(test_x_2) 

x_train.shape, y_train.shape, x_test.shape


((7661, 1380, 10), (7661, 120, 10), (535, 1380, 10))

In [4]:
def plot_series(x_series, y_series, y_predicted):
    #입력 series와 출력 series를 연속적으로 연결하여 시각적으로 보여주는 코드 입니다.
    plt.plot(x_series, label = 'input_series')
    plt.plot(np.arange(len(x_series), len(x_series)+len(y_series)),
             y_series, label = 'actual_series') 
    plt.plot(np.arange(len(x_series), len(x_series)+len(y_predicted)),
             y_predicted, label = 'predicted_series') 
    #plt.axhline(1, c = 'red')
    plt.legend()


In [5]:
def plot_predicted_series(x_series, y_predicted):
    #입력 series와 출력 series를 연속적으로 연결하여 시각적으로 보여주는 코드 입니다.
    plt.plot(x_series, label = 'input_series')
    plt.plot(np.arange(len(x_series), len(x_series)+len(y_predicted)),
             y_predicted, label = 'predicted_series') 
    #plt.axhline(1, c = 'red')
    plt.legend()


In [6]:
OC = np.concatenate([x_train, y_train], axis = 1) 
OC.shape


(7661, 1500, 10)

In [7]:
OC = OC[:,:,[1,4]] 
OC.shape


(7661, 1500, 2)

# Preprocess Data

This time we try a classification version. 
Given a sequence (t1,...,tN), is the price at tN+K greater than the buy price? 

In [11]:
K = 120 
N = 30
seq_len = 1500 
features = 2
X = [] 
Y = [] 

for j in tqdm(range(OC.shape[0]), position = 0, leave = True): 
    i = 0
    while i+N+K < 1500: 
        X.append(OC[j, i:i+N, :]) 
        if OC[j, i+N+K, 0] > 1: # not taking commission into account for now 
            Y.append(1.0) 
        else: 
            Y.append(0.0) 
        i += 1   
        
        
X = np.asarray(X) 
Y = np.asarray(Y)

X.shape, Y.shape

100%|██████████| 7661/7661 [00:13<00:00, 574.39it/s]


((10342350, 30, 2), (10342350,))

# Define Model

In [13]:
class Time2Vector(Layer):
    def __init__(self, seq_len, **kwargs):
        super(Time2Vector, self).__init__()
        self.seq_len = seq_len

    def build(self, input_shape):
        '''Initialize weights and biases with shape (batch, seq_len)'''
        self.weights_linear = self.add_weight(name='weight_linear',
                                              shape=(int(self.seq_len),),
                                              initializer='glorot_uniform',
                                              trainable=True)
    
        self.bias_linear = self.add_weight(name='bias_linear',
                                           shape=(int(self.seq_len),),
                                           initializer='glorot_uniform',
                                           trainable=True)
    
        self.weights_periodic = self.add_weight(name='weight_periodic',
                                                shape=(int(self.seq_len),),
                                                initializer='glorot_uniform',
                                                trainable=True)

        self.bias_periodic = self.add_weight(name='bias_periodic',
                                             shape=(int(self.seq_len),),
                                             initializer='glorot_uniform',
                                             trainable=True)

    def call(self, x):
        '''Calculate linear and periodic time features'''
        x = tf.math.reduce_mean(x[:,:,:], axis=-1) 
        time_linear = self.weights_linear * x + self.bias_linear # Linear time feature
        time_linear = tf.expand_dims(time_linear, axis=-1) # Add dimension (batch, seq_len, 1)
    
        time_periodic = tf.math.sin(tf.multiply(x, self.weights_periodic) + self.bias_periodic)
        time_periodic = tf.expand_dims(time_periodic, axis=-1) # Add dimension (batch, seq_len, 1)
        return tf.concat([time_linear, time_periodic], axis=-1) # shape = (batch, seq_len, 2)
   
    def get_config(self): # Needed for saving and loading model with custom layer
        config = super().get_config().copy()
        config.update({'seq_len': self.seq_len})
        return config


In [16]:
def transformer_block(inputs, node, drop_rate, activation): 
    attn_output = MultiHeadAttention(num_heads = 4, key_dim = node)(inputs, inputs) 
    attn_output = Dropout(drop_rate)(attn_output) 
    out1 = LayerNormalization(epsilon=1e-6)(inputs + attn_output) 
    ffn_output = Dense(node, activation = activation)(out1) 
    ffn_output = Dense(node)(ffn_output) 
    ffn_output = Dropout(drop_rate)(ffn_output)
    out2 = LayerNormalization(epsilon=1e-6)(out1 + ffn_output) 
    return out2
    
    
def build_transformer(node = 64, activation = 'relu', drop_rate = 0.2, num_layers = 3): 
    time_embedding = Time2Vector(N)
    inputs = Input((N, features))
    bn = BatchNormalization()(inputs) 
    x = time_embedding(bn) 
    x = Concatenate()([bn, x]) 
    x = Conv1D(node*2, 5, activation = activation)(x) 
    x = MaxPooling1D(3)(x) 
    x = Dropout(drop_rate)(x) 
    x = Conv1D(node, 5, activation = activation)(x) 
    x = MaxPooling1D(3)(x) 
    x = Dropout(drop_rate)(x) 
    
    positions = tf.range(start=0, limit=x.shape[1], delta=1)
    positions = Embedding(input_dim = x.shape[1], output_dim = node)(positions) 
    x = x + positions 
    
    for i in range(num_layers): 
        x = transformer_block(x, node, drop_rate, activation)
        
    x = GlobalMaxPooling1D()(x)
    x = Dropout(drop_rate)(x)
    x = Dense(1, activation = 'sigmoid')(x) 
    model = Model(inputs=inputs,outputs=x) 
    model.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics=['accuracy']) 
    return model


In [17]:
model = build_transformer()
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 30, 2)]      0                                            
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 30, 2)        8           input_2[0][0]                    
__________________________________________________________________________________________________
time2_vector_1 (Time2Vector)    (None, 30, 2)        120         batch_normalization_1[0][0]      
__________________________________________________________________________________________________
concatenate_1 (Concatenate)     (None, 30, 4)        0           batch_normalization_1[0][0]      
                                                                 time2_vector_1[0][0]       

In [18]:
model_path = 'TRANSFORMER_OC_time2vec_classification_epoch_{epoch:03d}_val_{val_loss:.3f}_acc_{val_accuracy:.3f}.h5'
learning_rate_reduction = ReduceLROnPlateau(monitor = 'val_loss', patience = 3, verbose = 1, factor = 0.5)
checkpoint = ModelCheckpoint(filepath = model_path, monitor = 'val_loss', verbose = 1, save_best_only = True)
early_stopping = EarlyStopping(monitor = 'val_loss', patience = 10) 


history = model.fit(X, 
                    Y, 
                    batch_size = 128, ## avoid OOM 
                    epochs = 50, 
                    callbacks = [learning_rate_reduction, checkpoint], 
                    validation_split = 0.1)


Epoch 1/50
72720/72720 [==============================] - 1005s 12ms/step - loss: 0.4576 - accuracy: 0.7810 - val_loss: 0.4220 - val_accuracy: 0.8032

Epoch 00001: val_loss improved from inf to 0.42200, saving model to TRANSFORMER_OC_time2vec_classification_epoch_001_val_0.422_acc_0.803.h5
Epoch 2/50
 3175/72720 [>.............................] - ETA: 13:03 - loss: 0.4513 - accuracy: 0.7828

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



42325/72720 [================>.............] - ETA: 5:44 - loss: 0.4511 - accuracy: 0.7828

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



72720/72720 [==============================] - 860s 12ms/step - loss: 0.4509 - accuracy: 0.7829 - val_loss: 0.4208 - val_accuracy: 0.8050

Epoch 00002: val_loss improved from 0.42200 to 0.42083, saving model to TRANSFORMER_OC_time2vec_classification_epoch_002_val_0.421_acc_0.805.h5
Epoch 3/50
72720/72720 [==============================] - 881s 12ms/step - loss: 0.4501 - accuracy: 0.7830 - val_loss: 0.4210 - val_accuracy: 0.8050

Epoch 00003: val_loss did not improve from 0.42083
Epoch 4/50
14755/72720 [=====>........................] - ETA: 10:53 - loss: 0.4499 - accuracy: 0.7835

...

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



72720/72720 [==============================] - 857s 12ms/step - loss: 0.4469 - accuracy: 0.7840 - val_loss: 0.4195 - val_accuracy: 0.8051

Epoch 00016: ReduceLROnPlateau reducing learning rate to 6.25000029685907e-05.

Epoch 00016: val_loss did not improve from 0.41942
Epoch 17/50
 6488/72720 [=>............................] - ETA: 12:25 - loss: 0.4459 - accuracy: 0.7842

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



44538/72720 [=================>............] - ETA: 5:16 - loss: 0.4469 - accuracy: 0.7839

KeyboardInterrupt: 

# Make Prediction

In [ ]:
best_model = load_model('', custom_objects = {'Time2Vector':Time2Vector})
best_model.summary()


In [ ]:
X_test = [] 
for j in tqdm(range(x_test.shape[0]), position = 0, leave = True): 
    for i in range(seq_len-K-N-120, seq_len-K-N):
        X_test.append(x_test[j, i:i+N, [1,4]])  

X_test = np.asarray(X_test).reshape((-1,N,features))
X_test.shape


In [ ]:
predicted = best_model.predict(X_test) 
predicted = predicted.reshape((-1,120)) 
predicted.shape

In [ ]:
## create submission dataframe version 1 
buy_quantities = [] 
sell_times = [] 

for i in tqdm(range(predicted.shape[0]), position = 0, leave = True): 
    sell_time = np.argmax(predicted[i,:]) 
    sell_times.append(sell_time) 
    cnt = 0 
    for j in range(120): 
        if predicted[i,j] > 0.5:    
            cnt += 1 
    buy_quantity = cnt/120 
    buy_quantities.append(buy_quantity)
    
submission.iloc[:,1] = buy_quantities 
submission.iloc[:,2] = sell_times  
submission.to_csv('transformers_OC_Open_classification.csv',index=False)
